# US Power Outage Analysis

**Name(s)**: Layth Marabeh, Khanh Phan, Danny Xia   
**Repository Link**: https://github.com/k-phantastic/US-Power-Outage-Analysis   
**Website Link**: https://github.com/k-phantastic/US-Power-Outage-Analysis

In [35]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from utils import * 

# For widescreen display
pd.set_option('display.max_columns', None)

## Step 1: Introduction

# CHECKPOINT 1: 
(2 points) Which of the three datasets did you choose? Why?

# Dataset Information from Purdue University 

**Source:** https://engineering.purdue.edu/LASCI/research-data/outages/outage.xlsx  
**Data Dictionary:** https://www.sciencedirect.com/science/article/pii/S2352340918307182?via%3Dihub#t0005

>This dataset includes the major outages witnessed by different states in the continental U.S. Besides major outages, this data contains information on geographical location of the outages, regional climatic information, land-use characteristics, electricity consumption patterns and economic characteristics of the states affected by the outages. 

> Column information is located in Table 1, Variable descriptions of the article

In [25]:
# Load the dataset
file_path = 'data/outage.xlsx'

raw_df = pd.read_excel(file_path)
raw_df.head(10)

,Major power outage events in the continental U.S.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56
0,Time period: January 2000 - July 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Regions affected: Outages reported in this dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,NaN,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,30.03,34.21,35.73,2345860,284978,9898,2640737,88.83,10.79,0.37,53499,49091,1.09,1.9,5226,291955,1.79,2.2,5457125,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
8,NaN,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,28.1,34.5,37.37,2300291,276463,10150,2586905,88.92,10.69,0.39,50447,47287,1.07,2.7,4571,267895,1.71,2.1,5310903,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
9,NaN,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,thunderstorm,NaN,2550,NaN,68200,11.79,9.25,6.71,9.19,1851519,1941174,1993026,5787064,31.99,33.54,34.44,2317336,278466,11010,2606813,88.9,10.68,0.42,51598,48156,1.07,0.6,5364,277627,1.93,2.2,5380443,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48


In [26]:
# Initial file has the header in row 5, with first column being blank
df = pd.read_excel(file_path, header=5, usecols=range(1, 57))

# Skip the units row
df = df.drop(index=0)
df.head()

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,NaN,NaN,3060,NaN,70000.0,11.6,9.18,6.81,9.28,2332915,2114774,2113291,6562520,35.55,32.23,32.2,2.31e+06,276286.0,10673.0,2.60e+06,88.94,10.64,0.41,51268,47586,1.08,1.6,4802,274182,1.75,2.2,5.35e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
2,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,30.03,34.21,35.73,2.35e+06,284978.0,9898.0,2.64e+06,88.83,10.79,0.37,53499,49091,1.09,1.9,5226,291955,1.79,2.2,5.46e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
3,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000.0,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,28.1,34.5,37.37,2.30e+06,276463.0,10150.0,2.59e+06,88.92,10.69,0.39,50447,47287,1.07,2.7,4571,267895,1.71,2.1,5.31e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
4,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,thunderstorm,NaN,2550,NaN,68200.0,11.79,9.25,6.71,9.19,1851519,1941174,1993026,5787064,31.99,33.54,34.44,2.32e+06,278466.0,11010.0,2.61e+06,88.9,10.68,0.42,51598,48156,1.07,0.6,5364,277627,1.93,2.2,5.38e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
5,5.0,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,NaN,NaN,1740,250,250000.0,13.07,10.16,7.74,10.43,2028875,2161612,1777937,5970339,33.98,36.21,29.78,2.37e+06,289044.0,9812.0,2.67e+06,88.82,10.81,0.37,54431,49844,1.09,1.7,4873,292023,1.67,2.2,5.49e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48


In [27]:
# DataFrame info
print(df.info())
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nShape: {df.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 1 to 1534
Data columns (total 56 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   OBS                      1534 non-null   float64
 1   YEAR                     1534 non-null   float64
 2   MONTH                    1525 non-null   float64
 3   U.S._STATE               1534 non-null   object 
 4   POSTAL.CODE              1534 non-null   object 
 5   NERC.REGION              1534 non-null   object 
 6   CLIMATE.REGION           1528 non-null   object 
 7   ANOMALY.LEVEL            1525 non-null   object 
 8   CLIMATE.CATEGORY         1525 non-null   object 
 9   OUTAGE.START.DATE        1525 non-null   object 
 10  OUTAGE.START.TIME        1525 non-null   object 
 11  OUTAGE.RESTORATION.DATE  1476 non-null   object 
 12  OUTAGE.RESTORATION.TIME  1476 non-null   object 
 13  CAUSE.CATEGORY           1534 non-null   object 
 14  CAUSE.CATEGORY.DETAIL   

## Step 2: Data Cleaning and Exploratory Data Analysis

# CHECKPOINT 1:

(6 points) Upload a screenshot of a plotly visualization you’ve created while completing Part 1, Step 2: Data Cleaning and Exploratory Data Analysis.

(6 points) What is the pair of hypotheses you plan on testing in Part 1, Step 4? What is the test statistic you plan on using?

(6 points) What is the column you plan on trying to predict in Part 1, Steps 5-8? Is it a classification or regression problem?


In [28]:
# TODO

## Step 3: Assessment of Missingness

In [29]:
# TODO

## Step 4: Hypothesis Testing

In [30]:
# TODO

## Step 5: Framing a Prediction Problem

In [31]:
# TODO

## Step 6: Baseline Model

In [32]:
# TODO

## Step 7: Final Model

In [33]:
# TODO

## Step 8: Fairness Analysis

In [34]:
# TODO